In [13]:
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input, Activation, Conv2D, Flatten, Dense, MaxPooling2D, Dropout, Add, LeakyReLU, UpSampling2D
from keras.models import Model, load_model
from keras.callbacks import ReduceLROnPlateau

from keras import Sequential # Add Sequential
import tensorflow as tf # for GPU set-up
import os

# Memory Allocation for GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## Load Dataset

In [8]:
DATA_PATH = os.path.abspath("..\\..\\00_MLDL\\00_BreadBrother\\04_Image_Segmentation\\") # Variable path

x_train = np.load(os.path.join(DATA_PATH, "dataset\\x_train.npy"))
y_train = np.load(os.path.join(DATA_PATH, "dataset\\y_train.npy"))
x_val = np.load(os.path.join(DATA_PATH, "dataset\\x_val.npy"))
y_val = np.load(os.path.join(DATA_PATH, "dataset\\y_val.npy"))

print(f"Training data shape: {x_train.shape}")
print(f"Validation data shape: {x_val.shape}")


Training data shape: (240, 256, 256, 1)
Validation data shape: (27, 256, 256, 1)


## Build Model

In [20]:
model = Sequential()
# Downsampling
model.add(Conv2D(32, kernel_size=3, activation="relu", padding="same",  input_shape=(256, 256, 1)))
model.add(MaxPooling2D(pool_size=2, padding="same")) # Reduce feature map size
model.add(Conv2D(64, kernel_size=3, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=2, padding="same"))
model.add(Conv2D(128, kernel_size=3, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=2, padding="same"))
model.add(Dense(128, activation="relu"))

# Upsampling
model.add(UpSampling2D(size=2))
model.add(Conv2D(128, kernel_size=3, activation="sigmoid", padding="same"))
model.add(UpSampling2D(size=2))
model.add(Conv2D(64, kernel_size=3, activation="sigmoid", padding="same"))
model.add(UpSampling2D(size=2))
model.add(Conv2D(1, kernel_size=3, activation="sigmoid", padding="same"))

# Compile
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc", "mse"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 256, 256, 32)      320       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 64, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 32, 32, 128)       0         
_________________________________________________________________
dense_8 (Dense)              (None, 32, 32, 128)      

## Training

In [19]:
%%time
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=32, callbacks=[
    ReduceLROnPlateau(monitor="val_loss", factor = 0.2, patience=10, verbose=1, mode="auto", min_lr=1e-05)
])

Train on 240 samples, validate on 27 samples
Epoch 1/100
240/240 [==============================] - 8s 33ms/step - loss: 0.6109 - acc: 0.6860 - mse: 0.2036 - val_loss: 0.5069 - val_acc: 0.7626 - val_mse: 0.1626
Epoch 2/100
240/240 [==============================] - 1s 4ms/step - loss: 0.4957 - acc: 0.7499 - mse: 0.1611 - val_loss: 0.4545 - val_acc: 0.7626 - val_mse: 0.1475
Epoch 3/100
240/240 [==============================] - 1s 5ms/step - loss: 0.4538 - acc: 0.7499 - mse: 0.1487 - val_loss: 0.4241 - val_acc: 0.7626 - val_mse: 0.1380
Epoch 4/100
240/240 [==============================] - 1s 4ms/step - loss: 0.4281 - acc: 0.7501 - mse: 0.1400 - val_loss: 0.3944 - val_acc: 0.7629 - val_mse: 0.1291
Epoch 5/100
240/240 [==============================] - 1s 4ms/step - loss: 0.4197 - acc: 0.7500 - mse: 0.1391 - val_loss: 0.4174 - val_acc: 0.7626 - val_mse: 0.1426
Epoch 6/100
240/240 [==============================] - 1s 5ms/step - loss: 0.3953 - acc: 0.7540 - mse: 0.1312 - val_loss: 0.3724 

KeyboardInterrupt: 

## Evaluation

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 7))

ax[0, 0].set_title('loss')
ax[0, 0].plot(history.history['loss'], 'r')
ax[0, 1].set_title('acc')
ax[0, 1].plot(history.history['acc'], 'b')

ax[1, 0].set_title('val_loss')
ax[1, 0].plot(history.history['val_loss'], 'r--')
ax[1, 1].set_title('val_acc')
ax[1, 1].plot(history.history['val_acc'], 'b--')